# Entire identification

This code contains the identification pipeline. It constructs a list of the materials used in NOMAD entries and attempts to find a PubChem CID for each material used.

In [6]:
import requests
import pickle
import os
import json
from groq import Groq # in NOMAD, needs to be installed first
import httpx
import api_keys # file containing a groq and elsevier api-key

# 1. Transform entries with ";" into lists
Some entries where multiple materials were used were separated using "," which makes them iterable lists. However, sometimes ";" was used as separator which is problematic as the entry will be treated as one list element instead of multiple. Therefore, the ";" entries need to be transformed into lists separated by ",".

In [7]:
# load dataframe with all cells
with open('data_RO1/df_all_ctls.pkl', 'rb') as f:
    df_all_ctls = pickle.load(f)

# Change ";" lists into proper lists separated by commas
for index, _ in df_all_ctls.iterrows():
    i = 0
    while i < len(df_all_ctls.at[index, "etl"]):
        if ";" in df_all_ctls.at[index, "etl"][i]:
            elements = df_all_ctls.at[index, "etl"][i].split(";")
            df_all_ctls.at[index, "etl"] = df_all_ctls.at[index, "etl"][:i] + elements + df_all_ctls.at[index, "etl"][i+1:]
        i += 1
        
for index, _ in df_all_ctls.iterrows():
    i = 0
    while i < len(df_all_ctls.at[index, "htl"]):
        if ";" in df_all_ctls.at[index, "htl"][i]:
            elements = df_all_ctls.at[index, "htl"][i].split(";")
            df_all_ctls.at[index, "htl"] = df_all_ctls.at[index, "htl"][:i] + elements + df_all_ctls.at[index, "htl"][i+1:]
        i += 1

# 2. Identify materials

For each material, the identification loop does the following things, leaving the loop once it has identified the material:
1. Try to get the material's CID in PubChem Compounds or Substances
2. Try retrieving the name from literature.

    2.1. Build a list of all papers where this material was used

        2.1.1. Build a list of DOIs where this material was used

        2.1.2. Try to get each paper from the Elsevier ScienceDirect API
    2.2. Let an LLM read each paper in search for the material's chemical name
3. With LLM's name suggestion, re-try identification in PubChem Compounds/Substances

In [8]:
def search_pubchem_by_name(industry_name, when_called = "initial"):
    '''
    This searches for a CTL material's CID in PubChem.
    Arguments:  industry_name (str) - the name of the material
                when_called (str) - "initial" or other; used to determine which 
                                    counter to increment
    Value: CID (int) - the CID of the material
    '''
    global id_in_comps, id_in_comps_LLM, id_in_subs, id_in_subs_LLM
    
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{industry_name}/cids/JSON"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        data = data['IdentifierList']['CID'][0]
        if when_called == "initial":
            print("SUCCESS: Initial compound search")
            id_in_comps = id_in_comps + 1
        else:
            print("SUCCESS: LLM result compound search")
            id_in_comps_LLM = id_in_comps_LLM + 1
        return data
    else:
        # if search in PubChem compounds fails, search in substances
        url_subs = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/name/{industry_name}/cids/JSON"

        response = requests.get(url_subs)

        if response.status_code == 200:
            data = response.json()
            data = data['InformationList']['Information'][0]['CID'][0]
            print("Substance search SUCCESSFUL")
            if when_called == "initial":
                print("SUCCESS: Initial substance search")
                id_in_subs = id_in_subs + 1
            else:
                print("SUCCESS: LLM result substance search")
                id_in_subs_LLM = id_in_subs_LLM + 1
            return data
        else:
            # if the search failed again, report failure
            print('debug: substance search unsuccessful')
            raise Exception(f"Error: Could not retrieve SID using the industry name alone. Status code: {response.status_code}")

## Functions needed for searching names in papers

In [9]:
def find_CID_in_text(industry_name, 
                     CTL_DOIs, 
                     paper_index = -1,
                     depth = 1):
    '''
    This function tries to find the chemical name in the paper text and to
    retrieve the CID from PubChem.
    Arguments: 
        industry_name (str) - the name of the material
        CTL_DOIs (list) - list of DOIs of papers with that CTL
        paper_index (int) - index of the paper in the list
    Value: 
        CID (int) - the CID of the material
    Dependencies: 
        paper_from_publisher, llm_retrieve_name, search_pubchem_by_name
    '''
    # necessary not to break execution due to too many recursions. 
    # Adjust to suit your machine's recursion limit.
    if depth > 2900:
        print(f'debug: Recursion limit reached at recusion = {depth}.')
        raise Exception('Recursion limit reached')

    print(f"debug: Recursion depth: {depth}")
    print(f'debug: length of CTL_DOIs: {len(CTL_DOIs)}. Paper Index: {paper_index}')
    
    # stop if all papers for the material have been searched
    if paper_index >= (len(CTL_DOIs) - 1):
        print('debug: unsucessfully searched through all papers associated with that material')
        raise Exception('All papers have been searched.')

    paper_index = paper_index + 1
    DOI = CTL_DOIs[paper_index]
    
    # Recursion section
    try:
        paper_text = paper_from_publisher(DOI)
    except:
        print('debug: paper_from_publisher failed. Recurring...')
        CID = find_CID_in_text(industry_name, CTL_DOIs, paper_index, depth+1)
        if CID is not None:
            return CID
    try:
        print('debug: trying llm_retrieve_name')
        chem_name = llm_retrieve_name(paper_text, industry_name)
    except:
        print('debug: llm_retrieve_name failed. Recurring...')
        CID = find_CID_in_text(industry_name, CTL_DOIs, paper_index, depth+1)
        if CID is not None:
            return CID
    try:
        print('debug: trying search_pubchem_by_name(chem_name)')
        CID = search_pubchem_by_name(chem_name, when_called="LLMresult")
        print('debug: search_pubchem_by_name with name found by LLM SUCCESSFUL!')
        return CID
    except:
        print('debug: trying search_pubchem_by_name(chem_name) failed. Recurring...')
        CID = find_CID_in_text(industry_name, CTL_DOIs, paper_index, depth+1)
        if CID is not None:
            return CID



def paper_from_publisher(DOI):
    '''
    This function tries publisher APIs to retrieve paper texts from ScienceDirect.
    Argument: DOI (str) - the DOI of the paper
    Value: paper_text (str) - the plain text of the paper
    '''

    def scidir_retrieve_paper(DOI, apikey):
        '''
        This function is used within paper_from_publisher to retrieve the paper
        text from ScienceDirect.
        '''
        apikey=apikey
        headers={
            "X-ELS-APIKey":apikey,
            "Accept":'application/json'
            }
        client = httpx.Client(headers=headers)
        query="&view=FULL"
        url=f"https://api.elsevier.com/content/article/doi/" + DOI
        r=client.get(url)
        print(f'debug: paper retrieval executed. This is the result: {r}')
        if r.status_code != 200:
            raise Exception(f"Error: The paper could not be found in ScienceDirect. Status code: {r.status_code}")
        return r

    # Get document
    try:
        scidir_response = scidir_retrieve_paper(DOI, api_keys.api_key_elsevier)

        json_acceptable_string = scidir_response.text
        d = json.loads(json_acceptable_string)
        return d['full-text-retrieval-response']['coredata']['dc:description']
    except:
        print("debug: Paper not found in ScienceDirect.")
        raise Exception("Error: Paper not found in ScienceDirect.")



def llm_retrieve_name(paper_text,
                      industry_name, 
                      api_key=api_keys.api_key_groq):
    '''
    This function retrieves the chemical name of the compound from the paper text.
    Arguments: 
        paper_text (str) - the text of the paper
        industry_name (str) - the name of the material
        api_key (str) - an api_key for groq
    Value: 
        chem_name (str) - the chemical name of the material
    '''
    groq = Groq(api_key=api_key)
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a solar cell scientist proficient in reading papers. You output only the chemical name of the compound asked for, nothing else.",
            },
            {
                "role": "user",
                "content": f"What is the chemical name pertaining to this abbreviation: {industry_name}? You can find it in this text: {paper_text}.",
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

In [10]:
def name_to_cid(industry_name, df):
    '''
    This function searches for the PubChem CID for materials. It tries to find
    the chemical name behind materials' industry name, searches for their CID
    and adds them to a dictionary. 
    Arguments:
        industry_name (str) - the name of the material
        df (pandas.DataFrame) - a dataframe with columns etl, htl and ref, 
        containing etl and htl materials and the DOI links to their papers
    Value:
        No value is returned but the name_to_CID_dict is updated.
    '''
    # skip if CID is none
    if industry_name == "none":
        return
    # skip if CID is already in the dictionary
    elif industry_name in name_to_CID_dict.keys() and name_to_CID_dict[industry_name] is not None:
        print('debug: material already in dictionary')
        return
    else:
        CID = None
        # procedure that tries to find the CID for the industry_name
        try:
            CID = search_pubchem_by_name(industry_name, when_called="initial")
            print('debug: initial search_pubchem_by_name SUCCESSFUL!')
            name_to_CID_dict[industry_name] = CID
            print('debug: CID added to dictionary')
            return            
        except:
            print('debug: initial search_pubchem_by_name did not work. Trying paper reading.')
            
            # build list of DOIs that mention the material
            CTL_DOIs = []
            df_nona = df.dropna(subset=['ref'])
            mask = df_nona['etl'].apply(lambda x: industry_name in x) | df_nona['htl'].apply(lambda x: industry_name in x)
            filtered_df = df_nona[mask]
            mask = filtered_df['ref'].str.contains('https://doi.org/')
            filtered_df = filtered_df[mask]
            CTL_DOIs = filtered_df['ref'].str.replace('https://doi.org/', '').tolist()
            CTL_DOIs = list(set(CTL_DOIs))

            if len(CTL_DOIs) == 0:
                print('debug: no references found in the dataframe for this material.')
            else:
                print('debug: list of papers successfully built')
            # go through the list of papers and try llm extraction
            try:
                CID = find_CID_in_text(industry_name, CTL_DOIs, depth=1)
            except Exception as e:
                if str(e) == 'All papers have been searched.':
                    print('debug: stopped recursion as all papers have been searched')
                    CID = None
                elif str(e) == 'Recursion limit reached':
                    print('debug: recursion limit of 2990 was reached.')
                    CID = None
            name_to_CID_dict[industry_name] = CID
            return

# Main Program

In [17]:
# Create a set of unique 'etl' and 'htl' materials
all_materials = df_all_ctls['etl'].sum() + df_all_ctls['htl'].sum()

# Create a set of unique materials
unique_materials = set([material.strip() for material in all_materials])

# initialize counters to count how materials were found
id_in_comps = 0
id_in_subs = 0
id_in_comps_LLM = 0
id_in_subs_LLM = 0

# initialize empty dictionary
name_to_CID_dict = {}

In [18]:
# Optional runtime "shortcut": Pre-define TiO2-c, which is used in many cells. 
# It does get identified by the loop but identification takes a very long time
# # as very many papers are read. This will cut runtime by several hours.
name_to_CID_dict['TiO2-c'] = 26042

In [19]:
# WARNING: The following may take a while, especially if you did not pre-define TiO2-c.

for index, material in enumerate(unique_materials):
    print(f"Next up: {material}. Progress: {index/len(unique_materials)}")
    name_to_cid(material, df_all_ctls)

Next up: ABA-SAM. Progress: 0.0
debug: substance search unsuccessful
debug: initial search_pubchem_by_name did not work. Trying paper reading.
debug: list of papers successfully built
debug: Recursion depth: 1
debug: length of CTL_DOIs: 1. Paper Index: -1


/tmp/ipykernel_4638/3285918931.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  CTL_DOIs = filtered_df['ref'].str.replace('https://doi.org/', '').tolist()


debug: paper retrieval executed. This is the result: <Response [404 Not Found]>
debug: Paper not found in ScienceDirect.
debug: paper_from_publisher failed. Recurring...
debug: Recursion depth: 2
debug: length of CTL_DOIs: 1. Paper Index: 0
debug: unsucessfully searched through all papers associated with that material
debug: stopped recursion as all papers have been searched
Next up: BaCoF4. Progress: 0.00040816326530612246
debug: substance search unsuccessful
debug: initial search_pubchem_by_name did not work. Trying paper reading.
debug: list of papers successfully built
debug: Recursion depth: 1
debug: length of CTL_DOIs: 1. Paper Index: -1


/tmp/ipykernel_4638/3285918931.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  CTL_DOIs = filtered_df['ref'].str.replace('https://doi.org/', '').tolist()


debug: paper retrieval executed. This is the result: <Response [404 Not Found]>
debug: Paper not found in ScienceDirect.
debug: paper_from_publisher failed. Recurring...
debug: Recursion depth: 2
debug: length of CTL_DOIs: 1. Paper Index: 0
debug: unsucessfully searched through all papers associated with that material
debug: stopped recursion as all papers have been searched
Next up: bis-C60. Progress: 0.0008163265306122449
debug: substance search unsuccessful
debug: initial search_pubchem_by_name did not work. Trying paper reading.
debug: list of papers successfully built
debug: Recursion depth: 1
debug: length of CTL_DOIs: 51. Paper Index: -1


...

debug: paper retrieval executed. This is the result: <Response [404 Not Found]>
debug: Paper not found in ScienceDirect.
debug: paper_from_publisher failed. Recurring...
debug: Recursion depth: 2
debug: length of CTL_DOIs: 1. Paper Index: 0
debug: unsucessfully searched through all papers associated with that material
debug: stopped recursion as all papers have been searched
Next up: PCBM-60-DPM-OE. Progress: 0.9987755102040816
debug: substance search unsuccessful
debug: initial search_pubchem_by_name did not work. Trying paper reading.
debug: no references found in the dataframe for this material.
debug: Recursion depth: 1
debug: length of CTL_DOIs: 0. Paper Index: -1
debug: unsucessfully searched through all papers associated with that material
debug: stopped recursion as all papers have been searched
Next up: Choline Chloride. Progress: 0.9991836734693877


/tmp/ipykernel_4638/3285918931.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  CTL_DOIs = filtered_df['ref'].str.replace('https://doi.org/', '').tolist()


SUCCESS: Initial compound search
debug: initial search_pubchem_by_name SUCCESSFUL!
debug: CID added to dictionary
Next up: 1,2-ethanedithiol (10 mM). Progress: 0.9995918367346939
debug: substance search unsuccessful
debug: initial search_pubchem_by_name did not work. Trying paper reading.
debug: list of papers successfully built
debug: Recursion depth: 1
debug: length of CTL_DOIs: 1. Paper Index: -1


/tmp/ipykernel_4638/3285918931.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  CTL_DOIs = filtered_df['ref'].str.replace('https://doi.org/', '').tolist()


debug: paper retrieval executed. This is the result: <Response [404 Not Found]>
debug: Paper not found in ScienceDirect.
debug: paper_from_publisher failed. Recurring...
debug: Recursion depth: 2
debug: length of CTL_DOIs: 1. Paper Index: 0
debug: unsucessfully searched through all papers associated with that material
debug: stopped recursion as all papers have been searched


In [22]:
# POTENTIALLY DANGEROUS! Don't overwrite the runtime-expensive dictionary file.
# ...unless you ran the above code to create it. Then absolutely save it as soon
# as possible.
with open('data_RO1/dict_all_cells_v2.pkl', 'wb') as f:
    pickle.dump(name_to_CID_dict, f)

# Result statistics

In [21]:
count = sum(value is not None for value in name_to_CID_dict.values())

print(f"Total of unique materials: {len(unique_materials)}")
print(f"Successfully identified: {count}")
print(f"Proportion: {count/len(unique_materials)}")
print(f"Identified in PubChem's Compound dictionary: {id_in_comps}")
print(f"Identified in PubChem's Substance dictionary: {id_in_subs}")
print(f"Identified in PubChem's Compound dictionary after LLM name extraction: {id_in_comps_LLM}")
print(f"Identified in PubChem's Substance dictionary after LLM name extraction: {id_in_subs_LLM}")
print("Please note that one additional entry - TiO2-c - was found using LLM extraction.")
print("However, it slowed the pipeline very much and was therefore replaced manually for repeated runs.")

Total of unique materials: 2450
Successfully identified: 809
Proportion: 0.3302040816326531
Identified in PubChem's Compound dictionary: 446
Identified in PubChem's Substance dictionary: 216
Identified in PubChem's Compound dictionary after LLM name extraction: 108
Identified in PubChem's Substance dictionary after LLM name extraction: 38
Please note that one additional entry - TiO2-c - was found using LLM extraction.
However, it slowed the pipeline very much and was therefore replaced manually for repeated runs.
